In [ ]:
# import libraries
import sys
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.utils import get_file_size
from compositionspace.io import get_reconstructed_positions, get_iontypes, get_iontypes_info
from compositionspace.clustering import ProcessClustering

In [ ]:
# ! pip list  
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

In [ ]:
RECONSTRUCTION_AND_RANGING = ("R21_08680-v02.pos", "R21_08680.rrng")
RECONSTRUCTION_AND_RANGING = ("PARAPROBE.Transcoder.Results.SimID.636502001.nxs",
                              "PARAPROBE.Ranger.Results.SimID.636502001.nxs")
config_file_path = f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml"
results_file_path = f"{MY_PROCESSED_DATA_PATH}/CompositionSpace.Results.nxs"

In [ ]:
from sklearn.cluster import DBSCAN

with h5py.File(results_file_path, "r") as h5r:
    target = 5
    phase_identifier = h5r[f"/entry1/segmentation/ic_opt/cluster_analysis{target}/y_pred"][:]
    all_vxl_pos = h5r["/entry1/voxelization/cg_grid/position"][:,:]
    print(all_vxl_pos[0:10, :])
    print(np.shape(all_vxl_pos))
    print(set(phase_identifier))
    
    eps = 2. * 0.5  # nm ! # self.params["ml_models"]["DBScan"]["eps"]
    min_samples = 5  # self.params["ml_models"]["DBScan"]["min_samples"]
    
    for target_phase in np.arange(0, np.max(tuple(set(phase_identifier))) + 1):
        print(f"Loop {target_phase}")
        # target_phase = 0  # <= np.max(phase_identifier)
        if target_phase > np.max(tuple(set(phase_identifier))):
            raise ValueError(f"Argument target_phase needs to be <= np.max(tuple(set(phase_identifier))) !")
        trg_vxl_pos = all_vxl_pos[phase_identifier == target_phase, :]
        print(np.shape(trg_vxl_pos))

        db = DBSCAN(eps=eps,
                    min_samples=min_samples,
                    metric="euclidean",
                    algorithm="kd_tree",
                    leaf_size=10,
                    p=None,
                    n_jobs=-1).fit(trg_vxl_pos)
        # print(np.unique(db.core_sample_indices_))
        # core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        # core_samples_mask[db.core_sample_indices_] = True
        # labels = db.labels_
        print(len(np.unique(db.labels_)))
        print(np.unique(db.labels_))
        del trg_vxl_pos

In [ ]:
Df_centroids_no_files, Df_centroids, Phase_columns = pdata.get_post_centroids(comps.voxel_centroid_output_file, 1)
print(f"df_centroids_no_files {Df_centroids_no_files}")
print(f"Df_centroids {Df_centroids}")
print(f"Phase_columns {Phase_columns}")

db = DBSCAN(eps=eps, min_samples=min_samples).fit(Df_centroids_no_files.values)  # eps=5., min_samples= 35
print(np.unique(db.core_sample_indices_))
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
pdata.DBSCAN_clustering(comps.voxel_centroid_output_file, cluster_id = 0,
                        plot=True, plot3d=True, save=True)

In [ ]:
segmentation = ProcessSegmentation(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=False)
segmentation.perform_pca_and_write_results()
segmentation.perform_bics_minimization_and_write_results()

In [ ]:
H5Web(results_file_path)

<div class="alert alert-block alert-danger">
Discussion points:<br>
* Modify NXapm_composition_space<br>
* What to show how to show?<br>
* Number of cluster vs Number of clusters? wording...?<br>
* Why to run the gm several times, ones in the bics loop ones in get composition cluster files 
</div>